# **Esercizi TensorFlow**
## *Esercitazione TensorFlow e Keras*

## **Es 11.0**

Scarichiamo il dataset *flowers*. Per farlo, usiamo il seguente codice:

In [1]:
import pathlib
from tensorflow import keras

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = keras.utils.get_file(origin=dataset_url, extract=True)
data_dir = pathlib.Path(data_dir).parent
data_dir = pathlib.Path(data_dir, 'flower_photos')

228813984/228813984 [==============================] - 29s 0us/step


Utilizziamo i metodi di Keras per caricare il dataset in memoria ed addestrare un modello per la classificazione di questo tipo:

Layer (type) | Output Shape | Param # 
:-----: | :--------: | :-----:
**rescaling_1 (Rescaling)**   |  (None, 150, 150, 3)  | 0
**conv_1 (Conv2D)**  |(None, 148, 148, 32)  | 896
**max_pool_1 (MaxPooling2D)**  |(None, 74, 74, 32)  | 0
**conv_2 (Conv2D)**   |   (None, 72, 72, 32)   | 9248
**max_pool_2 (MaxPooling2D)**  |  (None, 36, 36, 32)   | 0
**dropout (Dropout)**  |   (None, 36, 36, 32)  | 0
**flatten_1 (Flatten)**   |  (None, 41472)  | 0
**classification (Dense)**  | (None, 5)    | 207365    

Inferiamo il *numero di classi* del dataset (ovvero la X nel precedente sommario) usando l'attributo `class_names` del dataset ottenuto da `image_dataset_from_directory`.

In [3]:
import pathlib
from tensorflow import keras

# Scarichiamo il dataset Flowers.
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = keras.utils.get_file(origin=dataset_url, extract=True)
# Selezioniamo la cartella dove sono presenti i dati. Di solito è ~\user\username\.keras\dataset.
data_dir = pathlib.Path(data_dir).parent
data_dir = pathlib.Path(data_dir, 'flower_photos')

In [4]:
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
SEED = 42

# Creiamo i dataset di training e validazione. Usiamo un validation
# split di 0.2 (quindi con rapporto 80/20).
train = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED)

val = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [5]:
# Mostriamo a schermo le classi presenti nel dataset usando
# l'attributo class_names.
print(f'Classi nel dataset: {train.class_names}')

Classi nel dataset: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [6]:
# Creiamo ed addestriamo un modello.
model = keras.Sequential(
    [
        keras.Input(shape=(150, 150, 3)),
        keras.layers.Rescaling(1./255),
        keras.layers.Conv2D(
            32,
            (3, 3),
            activation='relu',
            name='conv_1'),
        keras.layers.MaxPooling2D(name='max_pool_1'),
        keras.layers.Conv2D(
            32,
            (3, 3),
            activation='relu',
            name='conv_2'),
        keras.layers.MaxPooling2D(name='max_pool_2'),
        keras.layers.Dropout(0.5),
        keras.layers.Flatten(),
        keras.layers.Dense(
            5,
            name='classification',
            activation='softmax')
    ]
)

# Copiamo il modello (ci servirà per il passo successivo)
model_callbacks = keras.models.clone_model(model)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=keras.metrics.SparseCategoricalAccuracy())

history = model.fit(
    train,
    validation_data=val,
    epochs=10)

Epoch 1/10
92/92 [==============================] - 25s 264ms/step - loss: 1.3072 - sparse_categorical_accuracy: 0.4251 - val_loss: 1.0861 - val_sparse_categorical_accuracy: 0.5627
Epoch 2/10
92/92 [==============================] - 26s 285ms/step - loss: 0.9858 - sparse_categorical_accuracy: 0.6151 - val_loss: 1.0254 - val_sparse_categorical_accuracy: 0.5926
Epoch 3/10
92/92 [==============================] - 27s 288ms/step - loss: 0.8284 - sparse_categorical_accuracy: 0.7033 - val_loss: 1.0012 - val_sparse_categorical_accuracy: 0.6104
Epoch 4/10
92/92 [==============================] - 28s 301ms/step - loss: 0.6542 - sparse_categorical_accuracy: 0.7735 - val_loss: 1.0452 - val_sparse_categorical_accuracy: 0.5940
Epoch 5/10
92/92 [==============================] - 30s 322ms/step - loss: 0.4895 - sparse_categorical_accuracy: 0.8341 - val_loss: 1.0266 - val_sparse_categorical_accuracy: 0.6376
Epoch 6/10
92/92 [==============================] - 27s 297ms/step - loss: 0.3490 - sparse_cate

In [7]:
# Riaddestriamo il modello utilizzando i callback.
model_callbacks.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=keras.metrics.SparseCategoricalAccuracy())

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_sparse_categorical_accuracy',
        min_delta=0.01,
        patience=3,
        restore_best_weights=True),
    keras.callbacks.TensorBoard()
]

history = model_callbacks.fit(
    train,
    validation_data=val,
    callbacks=callbacks,
    epochs=10)

Epoch 1/10
92/92 [==============================] - 29s 310ms/step - loss: 1.3601 - sparse_categorical_accuracy: 0.4172 - val_loss: 1.1183 - val_sparse_categorical_accuracy: 0.5109
Epoch 2/10
92/92 [==============================] - 28s 296ms/step - loss: 1.0638 - sparse_categorical_accuracy: 0.5848 - val_loss: 1.1016 - val_sparse_categorical_accuracy: 0.5354
Epoch 3/10
92/92 [==============================] - 27s 295ms/step - loss: 0.8942 - sparse_categorical_accuracy: 0.6683 - val_loss: 1.0759 - val_sparse_categorical_accuracy: 0.5845
Epoch 4/10
92/92 [==============================] - 28s 302ms/step - loss: 0.7047 - sparse_categorical_accuracy: 0.7476 - val_loss: 1.0143 - val_sparse_categorical_accuracy: 0.6022
Epoch 5/10
92/92 [==============================] - 27s 295ms/step - loss: 0.5479 - sparse_categorical_accuracy: 0.8127 - val_loss: 1.1474 - val_sparse_categorical_accuracy: 0.5817
Epoch 6/10
92/92 [==============================] - 27s 296ms/step - loss: 0.4504 - sparse_cate

## **Es 11.1**

Scarichiamo il dataset *Stack Overflow* e utilizziamo i metodi di Keras per caricare il dataset in memoria ed addestrare un modello per la classificazione di questo tipo:

Layer (type) | Output Shape | Param # 
:-----: | :--------: | :-----:
**dense_1 (Dense)**   |(None, 64) | 64064
**dense_2 (Dense)**  |(None, 4)   | 260

Usiamo gli stessi *callback* utilizzati in precedenza.

Inferiamo il numero di classi del dataset (ovvero la X nel precedente sommario) usando l'attributo `class_names` del dataset ottenuto da `text_dataset_from_directory`.

In [13]:
import pathlib
from tensorflow import keras
import tensorflow as tf

In [8]:
# Scarichiamo e salviamo il dataset Stack Overflow.
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"
data_dir = keras.utils.get_file(origin=dataset_url,
                            extract=True,
                            cache_subdir='datasets/stack_overflow')
# Selezioniamo la cartella dove sono presenti i dati. 
# Di solito è ~\user\username\.keras\dataset.
# Da notare che questo dataset ha al suo interno due cartelle,
# una di train ed una di test. Nel nostro esercizio utilizzeremo
# soltanto quella di train.
data_dir = pathlib.Path(data_dir).parent
train_dir = pathlib.Path(data_dir, 'train')

6053168/6053168 [==============================] - 1s 0us/step


In [9]:
BATCH_SIZE = 32
SEED = 42

# Creiamo i datset di training e validazione. Usiamo un validation
# split di 0.2 (quindi con rapporto 80/20).
train = keras.utils.text_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset='training',
    batch_size=BATCH_SIZE,
    seed=SEED)

val = keras.utils.text_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset='validation',
    batch_size=BATCH_SIZE,
    seed=SEED)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [10]:
# Mostriamo a schermo le classi presenti nel dataset usando
# l'attributo class_names.
print(f'Classi nel dataset: {train.class_names}')

Classi nel dataset: ['csharp', 'java', 'javascript', 'python']


In [11]:
# Creiamo un layer di tipo TextVectorization.
VOCAB_SIZE = 1000
vectorize_layer = keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')
# Chiamiamo il metodo adapt sul testo da caratterizzare.
train_text = train.map(lambda text, _: text)
vectorize_layer.adapt(train_text)

In [14]:
# Integriamo il layer di vettorizzazione all'interno di un
# modello di tipo sequenziale.
model = keras.models.Sequential()
model.add(
    keras.Input(shape=(1,),
    dtype=tf.string))
model.add(
    vectorize_layer)
model.add(
    keras.layers.Dense(
        64,
        activation='relu',
        name='dense_1'))
model.add(
    keras.layers.Dense(
        4,
        activation='softmax',
        name='dense_2'))

model.compile(optimizer='adam',
            loss=keras.losses.SparseCategoricalCrossentropy(),
            metrics=keras.metrics.SparseCategoricalAccuracy())

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_sparse_categorical_accuracy',
        min_delta=0.01,
        patience=3,
        restore_best_weights=True),
    keras.callbacks.TensorBoard()
]

history=model.fit(
    train,
    validation_data=val,
    epochs=10)

Epoch 1/10
200/200 [==============================] - 3s 11ms/step - loss: 0.9195 - sparse_categorical_accuracy: 0.6394 - val_loss: 0.6140 - val_sparse_categorical_accuracy: 0.7594
Epoch 2/10
200/200 [==============================] - 2s 10ms/step - loss: 0.5412 - sparse_categorical_accuracy: 0.7906 - val_loss: 0.5375 - val_sparse_categorical_accuracy: 0.7844
Epoch 3/10
200/200 [==============================] - 2s 10ms/step - loss: 0.4415 - sparse_categorical_accuracy: 0.8314 - val_loss: 0.5269 - val_sparse_categorical_accuracy: 0.7912
Epoch 4/10
200/200 [==============================] - 2s 10ms/step - loss: 0.3865 - sparse_categorical_accuracy: 0.8537 - val_loss: 0.5429 - val_sparse_categorical_accuracy: 0.7837
Epoch 5/10
200/200 [==============================] - 2s 10ms/step - loss: 0.3445 - sparse_categorical_accuracy: 0.8722 - val_loss: 0.5603 - val_sparse_categorical_accuracy: 0.7794
Epoch 6/10
200/200 [==============================] - 2s 10ms/step - loss: 0.3094 - sparse_cate